In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
color = sns.color_palette()
import warnings
warnings.filterwarnings('ignore')
# Garbage Collector to free up memory
import gc                         
gc.enable()   


# **Load the datasets**

In [ ]:
# Load the datasets
orders = pd.read_csv('../input/dataset/orders.csv' )
order_products_train = pd.read_csv('../input/dataset/order_products__train.csv')
order_products_prior = pd.read_csv('../input/dataset/order_products__prior.csv')
products = pd.read_csv('../input/dataset/products.csv')
aisles = pd.read_csv('../input/dataset/aisles.csv')
departments = pd.read_csv('../input/dataset/departments.csv')

**Inspecting orders dataset**

In [ ]:
# Get heads for the orders dataset
orders.head()

In [ ]:
# Get shape of orders dataset
orders.shape

In [ ]:
# Get a brief descriptive info on orders
orders.info()

In [ ]:
# Get missing values in orders dataset
orders.isnull().sum()

We have observed that there are 206209 missing values in days_since_prior_order column.

**Inspecting order_products_train dataset**

In [ ]:
# Get heads for the order_products_train dataset
order_products_train.head()

In [ ]:
# Get shape of order_products_train dataset
order_products_train.shape

In [ ]:
# Get missing values in order_products_train dataset
order_products_train.isnull().sum()

**Inspecting order_products_prior dataset**

In [ ]:
# Get head for order_products_prior
order_products_prior.head()

In [ ]:
# Get shape for order_products_prior
order_products_prior.shape

In [ ]:
# Get missing value for order_products_prior
order_products_prior.isnull().sum()

**Inspect products dataset**

In [ ]:
# Get heads for the products dataset
products.head()

In [ ]:
#  Get shape for products
products.shape

In [ ]:
# Get missing value for products
products.isnull().sum()

**Inspecting aisles dataset**

In [ ]:
# Get head for aisles
aisles.head()

In [ ]:
# Get shape for aisles
aisles.shape

In [ ]:
# Check missing values in aisle
aisles.isnull().sum()

**Inspecting departments dataset**

In [ ]:
# Get head for departments
departments.head()

In [ ]:
# Get shape for departments
departments.shape

In [ ]:
# Check missing values in departments
departments.isnull().sum()

# **EDA**

In [ ]:
# Get the number of orders in each days of a week
plt.figure(figsize=(6,4))
sns.countplot(x="order_dow", data=orders, color=color[0])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Day of week', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Orders by week day", fontsize=15)
plt.show()

The number of orders on weekends is more compared to weekdays as people stay at home and might have wanted to enjoy the foods.

In [ ]:
# Get the number of orders for each hour in a day
plt.figure(figsize=(6,4))
sns.countplot(x="order_hour_of_day", data=orders, color=color[0])
plt.ylabel('Count', fontsize=12)
plt.xlabel('Hour of day', fontsize=12)
plt.xticks(rotation='vertical')
plt.title("Orders by Hour of day", fontsize=15)
plt.show()

Peak hours wheremaximum orders are done is between 9 AM- 5PM. Less orders are placed before 7AM and after 11 PM.

In [ ]:
# Analyse how people orders a new one since last order
plt.figure(figsize=(10,6))
sns.countplot(orders['days_since_prior_order'])
plt.xticks(rotation=90)
plt.show()

Maximum number of users order again after 1 month. People also order after a week and this forms the second largest order habit.

In [ ]:
# Merge products and departments dataframes and then merging with aisles
products_details = pd.merge(left=products,right=departments,how="left")
products_details = pd.merge(left=products_details,right=aisles,how="left")
products_details.head()

In [ ]:
# Get the number of products in each department
plt.figure(figsize=(10,6))
g=sns.countplot(x="department",data=products_details)
g.set_xticklabels(g.get_xticklabels(), rotation=40, ha="right")
plt.show()

Personal care is the most abundant type of department available followed by snacks.

In [ ]:
# Get top 10 aisle with most number of products
plt.figure(figsize=(10,6))
top10_aisle=products_details["aisle"].value_counts()[:10].plot(kind="bar",title='Aisles')

missing is the aisle with most products available.

In [ ]:
# Merge order_products_train and products dataframes
order_products_name_train = pd.merge(left=order_products_train,right=products.loc[:,["product_id","product_name"]],on="product_id",how="left")

In [ ]:
# Get the top 10 common products which are most bought by the customers
common_Products=order_products_name_train[order_products_name_train.reordered == 1]["product_name"].value_counts().to_frame().reset_index()
plt.figure(figsize=(12,7))
plt.xticks(rotation=90)
sns.barplot(x="product_name", y="index", data=common_Products.head(10))
plt.ylabel('product_name', fontsize=12)
plt.xlabel('count', fontsize=12)
plt.show()

Banana is the most common type of product bought by people followed by Bag of organic banana. 

In [ ]:
# Merge order_products_name_train and products_details dataframes
order_products_name_train = pd.merge(left=order_products_name_train,right=products_details.loc[:,["product_id","aisle","department"]],on="product_id",how="left")

In [ ]:
#  Get the aisles which have most number of sales
common_aisle=order_products_name_train["aisle"].value_counts().to_frame().reset_index()
plt.figure(figsize=(12,7))
plt.xticks(rotation=90)
sns.barplot(x="aisle", y="index", data=common_aisle.head(10),palette="Blues_d")
plt.ylabel('aisle', fontsize=12)
plt.xlabel('count', fontsize=12)
plt.show()

Fresh vegetable aisle has the highest number of sales followed by fresh_fruits.

In [ ]:
#  Get the departments which have most number of sales
common_aisle=order_products_name_train["department"].value_counts().to_frame().reset_index()
plt.figure(figsize=(12,7))
plt.xticks(rotation=90)
sns.barplot(x="department", y="index", data=common_aisle,palette="Blues_d")
plt.ylabel('department', fontsize=12)
plt.xlabel('count', fontsize=12)
plt.show()


produce and dairy eggs are the top 2 departments with the highest number of sales.

In [ ]:
# Get the products which were reordered in each order from the train data.
train_data_reordered = order_products_train.groupby(["order_id","reordered"])["product_id"].apply(list).reset_index()
train_data_reordered = train_data_reordered[train_data_reordered.reordered == 1].drop(columns=["reordered"]).reset_index(drop=True)
train_data_reordered.head()

# **Feature Engineering**

In [ ]:
# Delete all unnncessary dataframes
del products_details
del order_products_name_train
del common_Products
del common_aisle
del train_data_reordered
gc.collect()

In [ ]:
# Get 15% of users as it is a huge dataset and will take lots of time to train.
orders = orders.loc[orders.user_id.isin(orders.user_id.drop_duplicates().sample(frac=0.15, random_state=101))] 

In [ ]:
# Convert character variables into category. 
aisles['aisle'] = aisles['aisle'].astype('category')
departments['department'] = departments['department'].astype('category')
orders['eval_set'] = orders['eval_set'].astype('category')
products['product_name'] = products['product_name'].astype('category')

In [ ]:
# Merge orders and order_products_prior datasets to get prior order dataset
prior_orders = pd.merge(orders, order_products_prior, on='order_id', how='inner')
prior_orders.head()

### Create Features using user_id

In [ ]:
# Create a feature based on number of orders placed by each user.
users = prior_orders.groupby(by='user_id')['order_number'].aggregate('max').to_frame('num_of_orders_for_each_user').reset_index()
users.head()

In [ ]:
# Get average number of products in  each orders placed by each users.

# Get the total number of products in each order.
toal_product_per_order = prior_orders.groupby(by=['user_id', 'order_id'])['product_id'].aggregate('count').to_frame('total_products_per_order').reset_index()

# Create a dataframe to get the average number of products purchased in  each orders by each user
avg_number_of_products_per_order = toal_product_per_order.groupby(by=['user_id'])['total_products_per_order'].mean().to_frame('avg_no_prd_per_order').reset_index()

# Delete unnecessay the toal_product_per_order dataframe
del [toal_product_per_order]
gc.collect()

#  Get head of avg_number_of_products_per_order
avg_number_of_products_per_order.head()

In [ ]:
# Importing the scipy's stats model
from scipy import stats

# Create a dataframe for the day of the week where users orders most
order_most_dow = prior_orders.groupby(by=['user_id'])['order_dow'].aggregate(lambda x : stats.mode(x)[0]).to_frame('dow_with_most_orders').reset_index()

# Get head of the dataset
order_most_dow.head()

In [ ]:
#Create a dataframe for hour of day where users has ordered most.
order_most_hod = prior_orders.groupby(by=['user_id'])['order_hour_of_day'].aggregate(lambda x : stats.mode(x)[0]).to_frame('hod_with_most_orders').reset_index()
order_most_hod.head()

In [ ]:
#Get a dataframe for reorder ratio of each user and set data type as float
user_reorder_ratio = prior_orders.groupby(by='user_id')['reordered'].aggregate('mean').to_frame('reorder_ratio').reset_index()
user_reorder_ratio['reorder_ratio'] = user_reorder_ratio['reorder_ratio'].astype(np.float16)
user_reorder_ratio.head()

In [ ]:
# Merging all the created user based features into the users dataset one by one.
users = users.merge(avg_number_of_products_per_order, on='user_id', how='left')
users = users.merge(order_most_dow, on='user_id', how='left')
users = users.merge(order_most_hod, on='user_id', how='left')
users = users.merge(user_reorder_ratio, on='user_id', how='left')

users.head()

In [ ]:
# Delete unnecessay dataframes
del [avg_number_of_products_per_order,order_most_dow,order_most_hod,user_reorder_ratio]
gc.collect()

### Create features using product_id.

In [ ]:
#Get a dataframe to show the number of times a product has been purchased.
purchased_num_of_times = prior_orders.groupby(by='product_id')['order_id'].aggregate('count').to_frame('purchased_num_of_times').reset_index()
purchased_num_of_times.head()


In [ ]:
#Get a dataframe for the reordered ratio for each product
product_reorder_ratio = prior_orders.groupby(by='product_id')['reordered'].aggregate('mean').to_frame('product_reorder_ratio').reset_index()
product_reorder_ratio.head()

In [ ]:
#Get a dataframe for avearage number of adding to cart for each product.
add_to_cart = prior_orders.groupby(by='product_id')['add_to_cart_order'].aggregate('mean').to_frame('product_avg_cart_addition').reset_index()
add_to_cart.head()

In [ ]:
# Merge all the created features based on product_id into the purchased_num_of_times dataset.
purchased_num_of_times = purchased_num_of_times.merge(product_reorder_ratio, on='product_id', how='left')
purchased_num_of_times = purchased_num_of_times.merge(add_to_cart, on='product_id', how='left')

#Delete unwanted dataframes.
del [product_reorder_ratio, add_to_cart]
gc.collect()

In [ ]:
# Get head of purchased_num_of_times
purchased_num_of_times.head()

### Creating features using user_id and product_id

In [ ]:
#Create a user_product dataframe which shows the number of times a user have bough a product.
user_product_data = prior_orders.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').to_frame('uxp_times_bought').reset_index()
user_product_data.head()

In [ ]:
#Create a dataframe  to find a product's order number when the user has bought a product for the first time.
product_first_order_num = prior_orders.groupby(by=['user_id', 'product_id'])['order_number'].aggregate('min').to_frame('first_order_number').reset_index()
product_first_order_num.head()

In [ ]:
#Get total number of orders by each user
total_orders = prior_orders.groupby('user_id')['order_number'].max().to_frame('total_orders').reset_index()
total_orders.head()

In [ ]:
# Merge total_orders and user_product_data dataframes to create a new dataframe user_product_df
user_product_df = pd.merge(total_orders, product_first_order_num, on='user_id', how='right')
user_product_df.head()

In [ ]:
# Calculate the order range.
# The +1 includes in the difference is the first order where the product has been purchased
user_product_df['order_range'] = user_product_df['total_orders'] - user_product_df['first_order_number'] + 1
user_product_df.head()

In [ ]:
#Create  a dataframe to show the number of times a user have bough a product.
number_of_times = prior_orders.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').to_frame('times_bought').reset_index()
number_of_times.head()

In [ ]:
# Merging number_of_times with user_product_df
uxp_ratio = pd.merge(number_of_times, user_product_df, on=['user_id', 'product_id'], how='left')
uxp_ratio.head()

In [ ]:
# Get a dataframe to calculate the reorder ratio for each product
uxp_ratio['uxp_reorder_ratio'] = uxp_ratio['times_bought'] / uxp_ratio['order_range']
uxp_ratio.head()

In [ ]:
#Drop all the unnecessary columns from uxp_ratio dataframe .
uxp_ratio.drop(['times_bought', 'total_orders', 'first_order_number', 'order_range'], axis=1, inplace=True)
uxp_ratio.head()

In [ ]:
#Merge uxp_ratio with user_product_data.
user_product_data = user_product_data.merge(uxp_ratio, on=['user_id', 'product_id'], how='left')

# Delete all unnecessay datasets
del [product_first_order_num, number_of_times,user_product_df,total_orders, uxp_ratio]
gc.collect()

In [ ]:
# Get head for user_product_data
user_product_data.head()

In [ ]:
#Create a column order_number_back to reverse the order number for each product in prior_orders dataframe
prior_orders['order_number_back'] = prior_orders.groupby(by=['user_id'])['order_number'].transform(max) - prior_orders.order_number + 1
prior_orders.head()

In [ ]:
#Update the dataframe to keep only the first 3 orders from the order_number_back.
temp_df = prior_orders.loc[prior_orders.order_number_back <= 3]
temp_df.head()

In [ ]:
#Get the products bought by users in the last 3 orders.
last_three_order = temp_df.groupby(by=['user_id', 'product_id'])['order_id'].aggregate('count').to_frame('uxp_last_three').reset_index()
last_three_order.head()

In [ ]:
#Get the ratio of the products bought in the last 3 orders.
last_three_order['uxp_ratio_last_three'] = last_three_order['uxp_last_three'] / 3
last_three_order.head()

In [ ]:
#Merge last_three_order with feature with uxp df.
user_product_data = user_product_data.merge(last_three_order, on=['user_id', 'product_id'], how='left')

# Delete unwanted dataframes
del [last_three_order, temp_df]
gc.collect()


In [ ]:
# Get the head of updated dataframe user_product_data.head()
user_product_data.head().head()

In [ ]:
# Check any missing values in user_product_data columns
user_product_data.isnull().sum()

In [ ]:
#Fill the NAN values with 0 in user_product_data.
user_product_data.fillna(0, inplace=True)

In [ ]:
# Confirm filling of missing values in user_product_data columns
user_product_data.isnull().sum()

### Create final dataframe for engineered features

In [ ]:
#  Merge user_product_data and users, then resulting dataframe with purchased_num_of_times to get featured_engineered_data dataset
featured_engineered_data = user_product_data.merge(users, on='user_id', how='left')
featured_engineered_data = featured_engineered_data.merge(purchased_num_of_times, on='product_id', how='left')

# Delete unncessary dataframes.
del [users, user_product_data, purchased_num_of_times]
gc.collect()

# Get head of featured_engineered_data
featured_engineered_data.head()

In [ ]:
# Check if any missing values found in featured_engineered_data dataframe columns
featured_engineered_data.isnull().sum()

## Creating Train and Test datasets

### Create training dataset

In [ ]:
# Keep only the future orders from all customers i.e. train  and test orders
orders_future = orders[((orders.eval_set=='train') | (orders.eval_set=='test'))]
orders_future = orders_future[['user_id', 'eval_set', 'order_id']]

In [ ]:
# merge the orders_future with featured_engineered_data to create a final dataframe.
final_data = featured_engineered_data.merge(orders_future, on='user_id', how='left')
final_data.head()

In [ ]:
#Create training data set.
train_data = final_data[final_data.eval_set=='train']
train_data.head()

In [ ]:
#Merge order_products__train  with into train_data datframe.
train_data = train_data.merge(order_products_train[['product_id', 'order_id', 'reordered']], on=['product_id', 'order_id'], how='left')
train_data.head()

In [ ]:
# Check if any missing values found in train_data dataframe columns
train_data.isnull().sum()

In [ ]:
# Fill the missing values in reordered column with 0
train_data['reordered'] = train_data['reordered'].fillna(0)
train_data.head()

In [ ]:
# Set user_id and product_id as the index of the train_data
train_data = train_data.set_index(['user_id', 'product_id'])

In [ ]:
# Drop unwanted columns from train_data dataframe
train_data = train_data.drop(['eval_set', 'order_id'], axis=1)

In [ ]:
# Get head of train_data
train_data.head()

### Create testing dataset

In [ ]:
#Keep only the future orders labelled as test
test_data = final_data[final_data.eval_set=='test']
test_data.head()

In [ ]:
# Set user_id and product_id as the index of the train_data
test_data = test_data.set_index(['user_id', 'product_id'])

# Drop unwanted columns from train_data dataframe
test_data = test_data.drop(['eval_set', 'order_id'], axis=1)

# Get head of test_data 
test_data.head()

In [ ]:
#Delete unnecessay dataframes
del [final_data, orders_future, products, order_products_train]
gc.collect()

## Building model using XGBoost and LGB

### XGBoost Model

In [ ]:
# Import libraries
import xgboost as xgb
from sklearn.model_selection import GridSearchCV

# Define X_train and y_train
X_train, y_train = train_data.drop('reordered', axis=1), train_data.reordered

# Set boosting parameters
parameters = {"max_depth":[5,8,10],
            "colsample_bytree":[0.3, 0.4]}

#  Fit the model after tuning with GridSearchCV
xgb_classifier = xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', num_boost_round=10)
gridsearch = GridSearchCV(xgb_classifier, parameters, cv=5, verbose=2, n_jobs=-1)
xgb_model = gridsearch.fit(X_train, y_train)

In [ ]:
# Get optimal model
xgb_optimal_model = gridsearch.best_estimator_

In [ ]:
# Get best params
xgb_optimal_model.get_params()

In [ ]:
# Get feature importance
xgb.plot_importance(xgb_optimal_model)

In [ ]:
# Get model predictions with threshold of 0.21 probability
test_prediction = (xgb_optimal_model.predict_proba(test_data)[:, 1] >= 0.21).astype(int)
test_prediction[:5]

In [ ]:
# Get model predictions on train set with threshold of 0.21 probability
train_prediction = (xgb_optimal_model.predict_proba(X_train)[:, 1] >= 0.21).astype(int)
train_prediction[:5]

In [ ]:
# Import evaluation matrices
from sklearn.metrics import f1_score, classification_report

In [ ]:
# Get f1 score and classification report
print(f'f1 Score: {f1_score(train_prediction, y_train)}')
print(classification_report(train_prediction, y_train))


In [ ]:
#Create the prediction as a new column in test_data
test_data['prediction'] = test_prediction
test_data.head()

In [ ]:
# Reset the index and create a dataset called final_df
final_df = test_data.reset_index()

# Keeping only the required columns to create  submission file
final_df = final_df[['product_id', 'user_id', 'prediction']]

# Collect garbage and show head of final_df
gc.collect()
final_df.head()

## Creating submission file

In [ ]:
#Create  a new dataframe orders_test 
orders_test = orders.loc[orders.eval_set == 'test', ['user_id', 'order_id']]
orders_test.head()

In [ ]:
#Merge  final_df with orders_test daatframe
final_df = final_df.merge(orders_test, on='user_id', how='left')
final_df.head()

In [ ]:
# Remove user_id column and convert product_id as integer
final_df = final_df.drop('user_id', axis=1)
final_df['product_id'] = final_df.product_id.astype(int)
final_df.head()

In [ ]:
# Create a dictionary to store product IDs whose reordered prediction value is 1
final_dict = dict()
for row in final_df.itertuples():
    if row.prediction== 1:
        try:
            final_dict[row.order_id] += ' ' + str(row.product_id)
        except:
            final_dict[row.order_id] = str(row.product_id)

# Update products whose reorder prediction value is 0 as None          
for order in final_df.order_id:
    if order not in final_dict:
        final_dict[order] = 'None'
 
# Collect garbage
gc.collect()

In [ ]:
# Convert the final_dict dictionary into a final submission dataframe called submission_df
submission_df = pd.DataFrame.from_dict(final_dict, orient='index')

# Reset index
submission_df.reset_index(inplace=True)

#Set column names
submission_df.columns = ['order_id', 'products']

# Get head
submission_df.head()

In [ ]:
# Create the final submission file
submission_df.to_csv('sub.csv', index=False, header=True)

In [ ]:
submission_df.shape